In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from zipfile import ZipFile

from urllib.request import urlopen
from bs4 import BeautifulSoup
from csv import writer
import re

In [2]:
#opening URL and getting HTTP response
url = "https://www.mohfw.gov.in/"
#url = "http://www.covid19india.com/"
html = urlopen(url)
soup = BeautifulSoup(html, 'lxml')

In [3]:
#fetch timing details for the data from the header tags
def listtostring(s):
    str1= ""
    for x in s:
        str1 +=x
        return str1

titles = []
for h2 in soup.find_all('h2'):
    titles.append(h2.text)
#timing details
reg_1 = re.compile(".*GMT")
temp_1 = list(filter(reg_1.match, titles))
time_date = listtostring(temp_1)

In [4]:
"""METHOD 1 - Using Beautiful soup to create Data Frame"""
# Table coloumn data and header
table = soup.table
table_row = table.find_all('tr')

row_data = []
for tr in table_row:
    td = tr.find_all('td')
    row = [dt.text for dt in td]
    row_data.append(row)

col_header_list = []
col_name = []
for tr in table_row:
    th = tr.find_all('th')
    head_row = [dh.text for dh in th]
    col_header_list.append(head_row)
col_header = col_header_list[0]
for col in col_header:
    col_name.append(col.strip())

In [5]:
"""METHOD 2 - Using read_html from pandas package to create Data Frame
dfs = pd.read_html(url)
for df in dfs:
    df
df"""

'METHOD 2 - Using read_html from pandas package to create Data Frame\ndfs = pd.read_html(url)\nfor df in dfs:\n    df\ndf'

In [6]:
# Data Frame
df_temp = pd.DataFrame(row_data, columns=col_name)
df_temp = df_temp.dropna(how='any',axis=0) 
ind_vir_df = df_temp.drop(columns='S. No.')
ind_vir_df.reset_index(drop=True,inplace=True)#from here data is exported

#Converting data type of coloumns to int and index is set
df_temp = ind_vir_df.set_index('Name of State / UT')
df_final = df_temp.astype(int)

In [7]:
#Exporting to .csv
#File path
file_path ='/Users/dinaeshs/Covid_19_India/data'
latest_data_states_file_path = '/Users/dinaeshs/Covid_19_India/data/latest_state_data.csv'
#File name
file_name_temp  = time_date.split(':',1)
file_name = file_name_temp[1].strip()
#File path and name
file_path_and_name = file_path+'/'+file_name+'.csv'
ind_vir_df.to_csv(file_path_and_name, index=False, header=True)
ind_vir_df.to_csv(latest_data_states_file_path, index=False, header=True)


In [8]:
#Adding new data series, calculated with other series
df_final['Active cases'] = df_final[df_final.columns[0]] - df_final[df_final.columns[1]] - df_final.Death
df_final.rename(columns={df_final.columns[0]:'Confirmed',df_final.columns[1]:'Cured'}, inplace=True)


In [9]:
df_final.loc['Total',:]= df_final.sum(axis=0)
df_final = df_final.astype(int)

In [10]:
#getting total values for the day and converting it to array
#date = file_name.split(',',1)[0]
day_total = df_final.iloc[-1:]
#day_total.loc[:,'Date'] = file_name
time_from_file = file_name.split(',')[1]
dt_time_str = file_name.split(',')[0] + " "+ time_from_file.split(' ')[1]
dt_time_str_formated = datetime.datetime.strptime(dt_time_str,'%d %B %Y %H:%M')
day_total.loc[:,'Date'] = dt_time_str_formated
final_count_for_day = day_total.iloc[0].values

/Users/dinaeshs/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/dinaeshs/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [11]:
#opeing all_data.csv file and storing the final total value with date and time
with open('/Users/dinaeshs/Covid_19_India/data/all_date.csv', 'a+', newline='') as write_obj:
    csv_writer = writer(write_obj)
    csv_writer.writerow(final_count_for_day)

In [12]:
# April 11th,2020 installed Kaggle API to pull data from sudalairajkumar/covid19-in-india
#!pip install kaggle
#command to install kaggleCLI in the the machine

In [13]:
# run this command to download the data
#!kaggle datasets download -d sudalairajkumar/covid19-in-india --force - to force download 
!kaggle datasets download -d sudalairajkumar/covid19-in-india

  0%|                                                | 0.00/114k [00:00<?, ?B/s]
100%|████████████████████████████████████████| 114k/114k [00:00<00:00, 5.73MB/s]


In [14]:
mv covid19-in-india.zip /Users/dinaeshs/Covid_19_India/data/

In [15]:
file_name_from_kaggle_dataset = '/Users/dinaeshs/Covid_19_India/data/covid19-in-india.zip'

with ZipFile(file_name_from_kaggle_dataset, 'r') as zf:
    zf.extractall('/Users/dinaeshs/Covid_19_India/data/kaggle_dataset')

In [16]:
df_temp.info()

<class 'pandas.core.frame.DataFrame'>
Index: 33 entries, Andaman and Nicobar Islands to West Bengal
Data columns (total 3 columns):
Total Confirmed cases (Including 76 foreign Nationals)    33 non-null object
Cured/Discharged/Migrated                                 33 non-null object
Death                                                     33 non-null object
dtypes: object(3)
memory usage: 1.0+ KB
